## Using Tensorflow like Numpy

### Tensors and Operations

In [1]:
import tensorflow as tf

In [2]:
tf.constant([[1, 2, 3], [4, 5, 5]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 5]])>

In [3]:
tf.constant(40) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=40>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape, t.dtype

(TensorShape([2, 3]), tf.float32)

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

#### Keras's Low Level API

In [11]:
K = tf.keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Tensors And NumPy

In [13]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [14]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [15]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [16]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [25]:
# Type Conversion

# tf.constant(2.) + tf.constant(40) # Gives Error
# tf.constant(2.) + tf.constant(40., dtype=tf.float64) # Also Give Error
tf.constant(2.0) + tf.cast(tf.constant(40.), dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

In [26]:
# Tensors can't be changed so we use something called

v = tf.Variable([[1., 2., 3.], [4. ,5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [27]:
# Modifying Tensors

v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [29]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   4.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

### Other Data Structures
* Sparse tensors(tf.SparseTensor) -> efficiently represent tensors using mostly zeroes.
* Tensor Array(tf.TensorArray) -> lists of tensors. all have same shape and same type. Could Be Dynamic
* Ragged Tensor(tf.RaggedTensor) -> same as Tensor Array but Static
* String Tensor(tf.string) -> they represent byte strings. can be converted into Unicode
* Sets(tf.sets) -> can be represented by regular tensor or sparse tensor
* Queues (tf.queue)

## Customizing Models and Training Algorithms

### Custom Loss Functions

In [1]:
import tensorflow as tf

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) * 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)


In [19]:
import numpy as np
X_train = np.arange(0, 100) * 3 + 2
X_train = X_train.reshape((100, 1)).astype(dtype=np.float32)
y_train = np.arange(0, 100, dtype=np.float32)

In [25]:
model = tf.keras.Sequential([tf.keras.layers.Dense(10, activation="relu"), 
                             tf.keras.layers.Dense(1)])
model.compile(loss=huber_fn, optimizer="nadam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, verbose=0)

In [26]:
model.save("test_model.h5")

### Saving and Loading Models That Contain Custom Components

In [27]:
model = tf.keras.models.load_model("test_model.h5", 
                                   custom_objects={"huber_fn": huber_fn})

In [29]:
# custom loss function with some different threshold

def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")
model.fit(X_train, y_train, epochs=10, verbose=0)
model.save("test_model2.h5")

In [30]:
# load model of custom threshold loss
model = tf.keras.models.load_model("test_model2.h5", 
                                   custom_objects={"huber_fn": create_huber(2.0)})

In [31]:
# Creating Loss function with keras.losses.Loss Subclass

class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}
    

In [34]:
model.compile(loss=HuberLoss(2.), optimizer="nadam")
model.fit(X_train, y_train, epochs=10, verbose=0)
model.save("test_model3.h5")

In [37]:
# model = tf.keras.models.load_model("test_model3.h5", 
#                                    custom_objects={"HuberLoss", HuberLoss})

### Custom Activation Function, Initializer, Regularizer, and Constraints

In [38]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0, tf.zeros_like(weights), weights)

In [39]:
layer = tf.keras.layers.Dense(30, activation=my_softplus, 
                              kernel_initializer=my_glorot_initializer, 
                              kernel_regularizer=my_l1_regularizer, 
                              kernel_constraint=my_positive_weights)

In [41]:
# custom l1 regualrizer with subclassing

class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(0.01 * weights))
    def get_config(self):
        return {"factor": self.factor}

### Custom  Metrics

In [42]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

In [43]:
precision = tf.keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [44]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [45]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [46]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [47]:
precision.reset_states()

In [48]:
# Custom Metrics using SubClassing

class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Custom layers

In [2]:
# custom layers with no weights
import tensorflow as tf
from tensorflow.keras import layers

layer = layers.Lambda(lambda x: tf.exp(x))

In [3]:
# custom dense layer with subclassing
class MyDense(layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units], 
                                      initializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
    
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, 
                "activation": tf.keras.activations.get(self.activation)}
    
        

In [4]:
# custom layer with multiple inputs

class MyMultiLayer(layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]
    
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1]


In [5]:
# custom layer with different behaivour while training
class MyGaussianNoise(layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        return X
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
    

### Custom Models

In [6]:
class ResidualBlock(layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [layers.Dense(n_neurons, activation="elu", 
                                    kernel_initializer="he_normal") for _ in range(n_layers)]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z
    

In [7]:
# custom model
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = layers.Dense(30, activation="elu", 
                                   kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = layers.Dense(output_dim)
    
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)
    

### Losses and Metrics Based on Model internals

In [8]:
class ReconstructionRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [layers.Dense(30, activation="selu", 
                                    kernel_initializer="lecun_normal") for _ in range(5)]
        self.out = layers.Dense(output_dim)
    
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[:-1]
        self.reconstruct = layers.Dense(n_inputs)
        super().build(batch_input_shape)
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)


In [9]:
import numpy as np
X = np.expand_dims(3 * np.arange(0, 100) + np.random.randn(100), axis=1)
y = np.arange(0, 100)

### Custom Training Loops

In [81]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.Sequential([
    layers.Dense(30, activation="elu", kernel_initializer="he_normal", 
                 kernel_regularizer=l2_reg), 
    layers.Dense(1, kernel_regularizer=l2_reg)
])

In [82]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [83]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

In [84]:
import numpy as np

X_train = np.expand_dims(3 * np.arange(0, 1000) + 4, axis=1)
y_train = np.arange(0, 1000)

In [85]:
X_train[:10], y_train[:10]

(array([[ 4],
        [ 7],
        [10],
        [13],
        [16],
        [19],
        [22],
        [25],
        [28],
        [31]]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [86]:
# hyperparameters
n_epochs = 20
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean()
metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [87]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Used When Using Kernel_constriants in layers while creating layer
        # for variable in model.variables:
        #   if variable.constraint is not None:
        #       variable.assign(variable.constraint(variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/20
1000/1000 - mean: 652224.5625 - mean_absolute_error: 390.40272
Epoch 2/20
1000/1000 - mean: 146244.1719 - mean_absolute_error: 123.4486
Epoch 3/20
1000/1000 - mean: 143371.0156 - mean_absolute_error: 120.3137
Epoch 4/20
1000/1000 - mean: 145454.6250 - mean_absolute_error: 124.0024
Epoch 5/20
1000/1000 - mean: 145320.7812 - mean_absolute_error: 123.1653
Epoch 6/20
1000/1000 - mean: 140095.8125 - mean_absolute_error: 120.6189
Epoch 7/20
1000/1000 - mean: 140602.5625 - mean_absolute_error: 119.2709
Epoch 8/20
1000/1000 - mean: 143449.7656 - mean_absolute_error: 120.5120
Epoch 9/20
1000/1000 - mean: 141065.4688 - mean_absolute_error: 118.8797
Epoch 10/20
1000/1000 - mean: 135523.9688 - mean_absolute_error: 113.4624
Epoch 11/20
1000/1000 - mean: 135005.6094 - mean_absolute_error: 113.3805
Epoch 12/20
1000/1000 - mean: 142380.9688 - mean_absolute_error: 117.3519
Epoch 13/20
1000/1000 - mean: 134118.3281 - mean_absolute_error: 113.9107
Epoch 14/20
1000/1000 - mean: 137253.5156 - me

## Tensorflow Functions and Graphs

In [92]:
cube = lambda x: x**3
cube(2)

8

In [93]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [94]:
# python function to tensorflow function
tf_cube = tf.function(cube)
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [95]:
# we can also use tensorflow decorator for this
@tf.function
def tf_cube(x): return x**3

In [97]:
# getting orginal python function from tensorflow function
tf_cube.python_function(2)

8

In [100]:
tf.autograph.to_code(cube)

"tf__lam = (lambda x: ag__.with_function_scope((lambda lscope: (x ** 3)), 'lscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)))\n"